In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


SystemError: ignored

In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# packages and modules from ml article linked here: https://www.pluralsight.com/guides/machine-learning-neural-networks-scikit-learn
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

# ignore some of above, trying a different model!
from sklearn.datasets import load_linnerud
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

# for classifier 
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression


In [5]:
def type_to_int(types):
    
    #type_to_int is a function takes a string parameter specifying either the type of the Pokemon or type of
    #move. This function converts the string to an integer for easier computation of type effectiveness
    #if the Pokemon does not have a 2nd type, the else statement returns -1
    
    if(type(types)==float):
        return -1
    elif types.lower() == "normal":
        return 0
    elif types.lower() == "fighting":
        return 1
    elif types.lower() == "flying":
        return 2
    elif types.lower() == "poison":
        return 3
    elif types.lower() == "ground":
        return 4
    elif types.lower() == "rock":
        return 5
    elif types.lower() == "bug":
        return 6
    elif types.lower() == "ghost":
        return 7
    elif types.lower() == "steel":
        return 8
    elif types.lower() == "fire":
        return 9
    elif types.lower() == "water":
        return 10
    elif types.lower() == "grass":
        return 11
    elif types.lower() == "electric":
        return 12
    elif types.lower() == "psychic":
        return 13
    elif types.lower() == "ice":
        return 14
    elif types.lower() == "dragon":
        return 15
    elif types.lower() == "dark":
        return 16
    elif types.lower() == "fairy":
        return 17
    else:
        return -1

In [6]:
url='https://drive.google.com/file/d/1VzeIrzna8SDkJCnIrCIxn5LXnpx3WcrA/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
poke = pd.read_csv(url) #import Pokemon data

url2='https://drive.google.com/file/d/1WR9i10kZ6bkI_7WV4fWmOXXNlaKgnBZ0/view?usp=share_link'
url2='https://drive.google.com/uc?id=' + url2.split('/')[-2]
training_data = pd.read_csv(url2) #import training data


poke = poke[poke.is_legendary == 0] #clean data for no legendaries
poke.reset_index(drop = True,inplace = True)
poke = poke.drop([131, 195, 196, 229, 348, 717]) #clean out problem children (Ditto, etc)
poke.reset_index(drop = True,inplace = True)

In [7]:
#zeroth row data point

pokemon_name = training_data.iloc[0][1] #pokemon 1 of our team is index 1 for column
pokemon_name_last = training_data.iloc[0][11] #last pokemon of opposing team is index 11 for column

#get desired stats
temp = poke.loc[poke["name"]==pokemon_name].squeeze()
#temp["name"],temp["hp"],[temp["type1"],temp["type2"]],temp["attack"],temp["defense"],temp["sp_attack"],temp["sp_defense"],temp["speed"]

In [28]:
training_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,29,30
0,0,Trevenant,Grimer,Sawk,Zorua,Tangela,Meowstic,Vulpix,Forretress,Wishiwashi,...,Hydreigon,Hypno,Delphox,Bisharp,Blastoise,Empoleon,Jolteon,Oranguru,Slowking,Kangaskhan
1,1,Arcanine,Tyrantrum,Venusaur,Budew,Kangaskhan,Braixen,Lickilicky,Victreebel,Espeon,...,Sneasel,Weavile,Greninja,Scizor,Slaking,Minior,Goodra,Ribombee,Lucario,Aerodactyl
2,2,Sawk,Butterfree,Poliwhirl,Tynamo,Seaking,Floette,Quagsire,Litten,Cloyster,...,Silvally,Seviper,Alakazam,Swalot,Oricorio,Delphox,Nidoqueen,Salazzle,Steelix,Typhlosion
3,3,Butterfree,Wurmple,Kangaskhan,Vibrava,Corsola,Toucannon,Hitmonlee,Ursaring,Karrablast,...,Greninja,Alakazam,Manectric,Jellicent,Absol,Mienshao,Porygon-Z,Arcanine,Aegislash,Chimecho
4,4,Elgyem,Mamoswine,Ambipom,Greninja,Caterpie,Toucannon,Magcargo,Stunky,Torracat,...,Purrloin,Liepard,Pansage,Simisage,Pansear,Simisear,Servine,Phione,Froslass,Gallade


In [33]:
training_data.iloc[3]

Unnamed: 0             3
0             Butterfree
1                Wurmple
2             Kangaskhan
3                Vibrava
4                Corsola
5              Toucannon
6              Hitmonlee
7               Ursaring
8             Karrablast
9                Turtwig
10                 Klink
11               Lucario
12             Metagross
13               Lampent
14               Slaking
15             Salamence
16             Gardevoir
17              Silvally
18              Garchomp
19               Lopunny
20             Klinklang
21              Greninja
22              Alakazam
23             Manectric
24             Jellicent
25                 Absol
26              Mienshao
27             Porygon-Z
28              Arcanine
29             Aegislash
30              Chimecho
Name: 3, dtype: object

In [9]:
X = training_data.iloc[:,1:12]
Y = training_data.iloc[:,12:-1 ]

In [10]:
# def extract_features(data, row, col):
#   features = np.zeros(21)
#   temp = poke.loc[poke["name"]==data.iloc[row, col]].squeeze()
#   features[type_to_int(temp["type1"])] = 1
#   if (type_to_int(temp["type2"]) >=0):
#     features[type_to_int(temp["type2"])] = 1
#   features[18] = temp["attack"]+temp["defense"]+temp["sp_attack"]+temp["sp_defense"]+temp["speed"]+temp["hp"]
#   features[19] = max(temp["attack"],temp["sp_attack"]) + temp["speed"]
#   features[20] = max(temp["defense"],temp["sp_defense"]) + temp["hp"]
#   return features

In [11]:
def extract_features(data, row, col):
  types = np.zeros(18)
  stats = np.zeros(3)
  temp = poke.loc[poke["name"]==data.iloc[row, col]].squeeze()
  types[type_to_int(temp["type1"])] = 1
  if (type_to_int(temp["type2"]) >=0):
    types[type_to_int(temp["type2"])] = 1
  stats[0] = temp["attack"]+temp["defense"]+temp["sp_attack"]+temp["sp_defense"]+temp["speed"]+temp["hp"]
  stats[1] = max(temp["attack"],temp["sp_attack"]) + temp["speed"]
  stats[2] = max(temp["defense"],temp["sp_defense"]) + temp["hp"]
  return types,stats

In [12]:
num_rows = len(training_data)
typesX = np.zeros((198,num_rows))
statsX = np.zeros((33,num_rows))

for j in range(num_rows):
  for i in range(11):
    typesX[i*18:(i*18+18), j], statsX[i*3:(i*3+3), j] = extract_features(X, j, i)

print(typesX[:,0], statsX[:,0])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0.] [474. 166. 167. 325. 105. 130. 465. 210. 150. 330. 145.  80. 435. 160.
 180. 466. 187. 155. 299. 115. 103. 465. 130. 215. 620. 170. 180. 535.
 220. 165. 355. 131. 122.]


In [13]:
typesY = np.zeros((18,num_rows))
statsY = np.zeros((3,num_rows))

for j in range(num_rows):
    typesY[:, j], statsY[:,j] = extract_features(Y, j, 0)

print(typesY[:,0], statsY[:,0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] [600. 325. 160.]


In [14]:
# don't need this right now, it's for a single output binary classifier (from previous reference article) TO DELETE
# X_train, X_test, y_train, y_test = train_test_split(featureX.transpose(), featureY.transpose(), test_size=0.30, random_state=40)
# print(X_train.shape); print(X_test.shape)

For our type output, this could be a binary classifer with classes 1 (is the type) and 0 (is not the type) with 18 outputs. 

For the stats predictor, we cannot use a classifier unless we had all possible classes (every integer option: not feasible), instead we should use a regression model?)

Idea: two different models, one for classifying on type and one for regression

Idea2: use entirely a regression model and pick the type 1/2 that are the closest to 1

In [15]:
#regr = MultiOutputRegressor(Ridge(random_state=123)).fit(featureX.transpose(), featureY.transpose())

In [16]:
#regr.predict(featureX.transpose()[[0]])

This worked kinda but also not! Stats seems like reasonable numbers for stats, but type doesn't make much sense. Try: two models, one for type (classifier multioutput) and one for stats (regressor multioutput) and then use both of these results and combine them for our total output :)

In [17]:
regr = MultiOutputRegressor(Ridge(random_state = 123)).fit(statsX.transpose(), statsY.transpose())

In [34]:
statsPredict=regr.predict(statsX.transpose())

In [35]:
statsPredict

array([[643.88147839, 250.54993431, 211.06853559],
       [645.92533108, 259.55498623, 201.49643557],
       [626.93805044, 248.83794125, 196.62193455],
       ...,
       [599.04693393, 234.27753667, 193.15007847],
       [642.12454779, 256.59643234, 196.54247578],
       [603.08779872, 219.23035738, 205.27514673]])

In [19]:
clf = MultiOutputClassifier(LogisticRegression()).fit(typesX.transpose(), typesY.transpose())

In [20]:
typesY.transpose()[-1:]

array([[0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [21]:
clf.predict(typesX.transpose()[-1:])

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

This binary multiclass output does not return all outputs as having a type. We really need it to return 1 or 2 types for our output pokemon (this is a BIG part of choosing pokemon to battle). New idea: make each type a class with an int 0-17, or 1-18, and maybe another class as a placeholder for a type when a pokemon has only 1 type (or simply have [18 18]. Then, use a multiclass classifier with an output of 2 (representing types)

Need to make data changes :'(

In [ ]:
pred = clf.predict(typesX.transpose())
np.where(np.sum(pred,axis=1) != 0)[0]

In [32]:
pred[3]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

MAYBE types doooooo only matter in 1/3 of our battles ?? (:

If type isn't  all zero, then condition on type and search for nearest neighbor of that type?
weight based on the team's deficiency
